In [ ]:
# Useful packages
import os
import numpy as np
import pandas as pd
import io
import json
import datetime as pkgdt
import time
import requests 
from requests.adapters import HTTPAdapter
import itertools # list operators
from tqdm import tqdm


In [ ]:

def page_gares(numero_page) :
    return requests.get(
        ('https://api.sncf.com/v1/coverage/sncf/stop_areas?start_page={}').format(numero_page),
        auth=('16cbfb01-1943-471f-aac9-7a1139abab77', ''))

######################################
# on commence par la première page qui nous donne le nombre de résultats par page ainsi que le nombre total de résultats

page_initiale = page_gares(0)
item_per_page = page_initiale.json()['pagination']['items_per_page']
total_items = page_initiale.json()['pagination']['total_result']
dfs = []

# on fait une boucle sur toutes les pages suivantes
print_done = {}

for page in range(int(total_items/item_per_page)+1) :
    stations_page = page_gares(page)

    ensemble_stations = stations_page.json()

    if 'stop_areas' not in ensemble_stations:
        # pas d'arrêt
        continue

    # on ne retient que les informations qui nous intéressent
    for station in ensemble_stations['stop_areas']:

        station['lat'] = station['coord']['lat']
        station["lon"]  = station['coord']['lon']

    stations = ensemble_stations['stop_areas']
    dp = pd.DataFrame(stations)

    dfs.append(dp)
    if page % 10 == 0:
        print("Page", page, "---", dp.shape)
        
gares = pd.concat(dfs)
gares.to_csv("./Data/ensemble_gares.csv")
print(gares.shape)
gares.head()



In [ ]:
def id_gare(nom_gare):
    if nom_gare=='montparnasse':
        return('87391003')
    elif nom_gare=='paris_nord':
        return('87271007')
    elif nom_gare=='lyon_partdieu':
        return('87723197')


temps = time.time()
print(temps)

In [ ]:
nom_gare_depart = 'paris_nord'

def page_departs(numero_page) :
    return requests.get(
        ('https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:'+id_gare(nom_gare_depart)+'/departures?datetime=20221027T105600').format(numero_page),
        auth=('16cbfb01-1943-471f-aac9-7a1139abab77', ''))

######################################
# on commence par la première page qui nous donne le nombre de résultats par page ainsi que le nombre total de résultats

page_initiale = page_departs(0)
item_per_page = page_initiale.json()['pagination']['items_per_page']
total_items = page_initiale.json()['pagination']['total_result']
departsdfs = []

# on fait une boucle sur toutes les pages suivantes
print_done = {}

for page in range(int(total_items/item_per_page)+1) :
    departs_page = page_departs(page)

    ensemble_departs = departs_page.json()

    if 'departures' not in ensemble_departs:
        # pas d'arrêt
        continue

    # on ne retient que les informations qui nous intéressent
    for departs in ensemble_departs['departures']:

        departs['label'] = departs['display_informations']['direction']
        

    departs = ensemble_departs['departures']
    dp= pd.DataFrame(departs)

    departsdfs.append(dp)
    if page % 10 == 0:
        print("je suis à la page", page, "---", dp.shape)
    
departs_liste = pd.concat(departsdfs)
departs_liste.head()

In [ ]:
departs_gares = pd.merge(departs_liste,gares,on='label',how='left')
departs_gares_loc= departs_gares[['label','lat','lon']]
departs_gares_loc.head()

In [ ]:
#Affichage Folium
import folium

departs_gares_latitude =departs_gares_loc["lat"]
departs_gares_longitude = departs_gares_loc["lon"]

departs_gares_latitude.head()

fmap= folium.Map(location=[departs_gares_latitude.iloc[1],departs_gares_longitude.iloc[1]])


n = departs_gares_loc.shape[0] #Nombre de gares
print(departs_gares_longitude.iloc[2])

for i in range(n):
    longitude = departs_gares_longitude.iloc[i]
    latitude = departs_gares_latitude.iloc[i]
    nom = departs_gares_loc['label'].iloc[i]
    if (not(pd.isna(latitude)) and not(pd.isna(longitude))):
   

        
        folium.Marker([latitude, longitude],
        popup=nom,
        icon=folium.Icon(color='red')).add_to(fmap)
fmap.save('Carte_SNCF_departs'+nom_gare_depart +'.html')
fmap